# Import and install libraries

In [13]:
# pip install pgmpy
# pip install scikit-fuzzy

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination


# Fuzzy Logic Implementation

This code implements a fuzzy logic system for assessing building damage based on various observed conditions, such as crack width, leaning angle, roof sagging, foundation cracks, and fire level. The system uses fuzzy logic to handle uncertainty and imprecise data, allowing it to make decisions even when the input data is not exact.


In [14]:

# Define the universe of variables
crack_width = np.arange(0, 15, 1)
leaning_angle = np.arange(0, 3, 0.1)
roof_sagging = np.arange(0, 10, 1)
foundation_cracks = np.arange(0, 15, 1)
fire_level = np.arange(0, 101, 1)  # Fire level in percentage (0% to 100%)
damage_severity = np.arange(0, 11, 1)

# Create fuzzy sets for crack width
minor_crack = fuzz.trimf(crack_width, [0, 0, 3])
moderate_crack = fuzz.trimf(crack_width, [2, 5, 10])
severe_crack = fuzz.trimf(crack_width, [8, 10, 15])

# Create fuzzy sets for leaning angle
minor_lean = fuzz.trimf(leaning_angle, [0, 0, 1])
moderate_lean = fuzz.trimf(leaning_angle, [0.5, 1, 1.5])
severe_lean = fuzz.trimf(leaning_angle, [1, 2, 3])

# Create fuzzy sets for roof sagging
minor_roof = fuzz.trimf(roof_sagging, [0, 0, 3])
moderate_roof = fuzz.trimf(roof_sagging, [2, 5, 8])
severe_roof = fuzz.trimf(roof_sagging, [6, 10, 10])

# Create fuzzy sets for foundation cracks
minor_foundation = fuzz.trimf(foundation_cracks, [0, 0, 5])
moderate_foundation = fuzz.trimf(foundation_cracks, [3, 7, 10])
severe_foundation = fuzz.trimf(foundation_cracks, [8, 10, 15])

# Create fuzzy sets for fire level
minor_fire = fuzz.trimf(fire_level, [0, 0, 30])
moderate_fire = fuzz.trimf(fire_level, [20, 50, 80])
severe_fire = fuzz.trimf(fire_level, [70, 100, 100])

# Create fuzzy sets for damage severity
minor_damage = fuzz.trimf(damage_severity, [0, 0, 4])
moderate_damage = fuzz.trimf(damage_severity, [2, 5, 8])
severe_damage = fuzz.trimf(damage_severity, [6, 10, 10])

# Define input and output variables
crack_input = ctrl.Antecedent(crack_width, 'crack_width')
lean_input = ctrl.Antecedent(leaning_angle, 'leaning_angle')
roof_input = ctrl.Antecedent(roof_sagging, 'roof_sagging')
foundation_input = ctrl.Antecedent(foundation_cracks, 'foundation_cracks')
fire_input = ctrl.Antecedent(fire_level, 'fire_level')
damage_output = ctrl.Consequent(damage_severity, 'damage_severity')

# Assign membership functions
crack_input['minor'] = minor_crack
crack_input['moderate'] = moderate_crack
crack_input['severe'] = severe_crack

lean_input['minor'] = minor_lean
lean_input['moderate'] = moderate_lean
lean_input['severe'] = severe_lean

roof_input['minor'] = minor_roof
roof_input['moderate'] = moderate_roof
roof_input['severe'] = severe_roof

foundation_input['minor'] = minor_foundation
foundation_input['moderate'] = moderate_foundation
foundation_input['severe'] = severe_foundation

fire_input['minor'] = minor_fire
fire_input['moderate'] = moderate_fire
fire_input['severe'] = severe_fire

damage_output['minor'] = minor_damage
damage_output['moderate'] = moderate_damage
damage_output['severe'] = severe_damage

# Define fuzzy rules
rule1 = ctrl.Rule(crack_input['minor'] & lean_input['minor'], damage_output['minor'])
rule2 = ctrl.Rule(crack_input['moderate'] | lean_input['moderate'], damage_output['moderate'])
rule3 = ctrl.Rule(crack_input['severe'] | lean_input['severe'], damage_output['severe'])
rule4 = ctrl.Rule(roof_input['severe'], damage_output['severe'])
rule5 = ctrl.Rule(roof_input['moderate'], damage_output['moderate'])
rule6 = ctrl.Rule(foundation_input['severe'], damage_output['severe'])
rule7 = ctrl.Rule(foundation_input['moderate'], damage_output['moderate'])
rule8 = ctrl.Rule(fire_input['severe'], damage_output['severe'])
rule9 = ctrl.Rule(fire_input['moderate'], damage_output['moderate'])

# Create control system
damage_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
damage_sim = ctrl.ControlSystemSimulation(damage_ctrl)


# Probabilistic Reasoning Implementation (Bayesian Network)

This code defines a Bayesian Network to model the relationships between various factors (e.g., crack width, leaning angle, roof sagging, foundation cracks, fire level) and the resulting structural damage. Bayesian networks are used to handle uncertainty and probabilistic reasoning.

In [15]:
# Define the Bayesian network structure
model = BayesianNetwork([('Crack_Width', 'Structural_Damage'),
                       ('Leaning_Angle', 'Structural_Damage'),
                       ('Roof_Sagging', 'Roof_Damage'),
                       ('Foundation_Cracks', 'Foundation_Damage'),
                       ('Fire_Level', 'Fire_Damage'),
                       ('Roof_Damage', 'Structural_Damage'),
                       ('Foundation_Damage', 'Structural_Damage'),
                       ('Fire_Damage', 'Structural_Damage')])

# Define conditional probability distributions (CPDs)
cpd_crack_width = TabularCPD(variable='Crack_Width', variable_card=3,
                             values=[[0.2], [0.5], [0.3]])

cpd_leaning_angle = TabularCPD(variable='Leaning_Angle', variable_card=3,
                               values=[[0.3], [0.4], [0.3]])

cpd_roof_sagging = TabularCPD(variable='Roof_Sagging', variable_card=3,
                              values=[[0.2], [0.5], [0.3]])

cpd_foundation_cracks = TabularCPD(variable='Foundation_Cracks', variable_card=3,
                                   values=[[0.2], [0.5], [0.3]])

cpd_fire_level = TabularCPD(variable='Fire_Level', variable_card=3,
                            values=[[0.2], [0.5], [0.3]])

cpd_roof_damage = TabularCPD(variable='Roof_Damage', variable_card=3,
                             values=[[0.8, 0.5, 0.2],
                                     [0.15, 0.4, 0.5],
                                     [0.05, 0.1, 0.3]],
                             evidence=['Roof_Sagging'],
                             evidence_card=[3])

cpd_foundation_damage = TabularCPD(variable='Foundation_Damage', variable_card=3,
                                   values=[[0.8, 0.5, 0.2],
                                           [0.15, 0.4, 0.5],
                                           [0.05, 0.1, 0.3]],
                                   evidence=['Foundation_Cracks'],
                                   evidence_card=[3])

cpd_fire_damage = TabularCPD(variable='Fire_Damage', variable_card=3,
                             values=[[0.8, 0.5, 0.2],
                                     [0.15, 0.4, 0.5],
                                     [0.05, 0.1, 0.3]],
                             evidence=['Fire_Level'],
                             evidence_card=[3])


cpd_structural_damage = TabularCPD(
    variable='Structural_Damage',
    variable_card=3,
    values=[
        # Minor Damage probabilities
        [0.8] * 81 + [0.1] * 162,
        # Moderate Damage probabilities
        [0.15] * 81 + [0.3] * 162,
        # Severe Damage probabilities
        [0.05] * 81 + [0.6] * 162
    ],
    evidence=['Crack_Width', 'Leaning_Angle', 'Roof_Damage', 'Foundation_Damage', 'Fire_Damage'],
    evidence_card=[3, 3, 3, 3, 3]
)

# Add CPDs to the model
model.add_cpds(cpd_crack_width, cpd_leaning_angle, cpd_roof_sagging,
               cpd_foundation_cracks, cpd_fire_level, cpd_roof_damage,
               cpd_foundation_damage, cpd_fire_damage, cpd_structural_damage)

# Perform inference
inference = VariableElimination(model)


# Integration of Both Approaches

This code combines fuzzy logic and probabilistic reasoning to assess building damage. The function takes observed conditions as input and evaluates the damage using both fuzzy logic and a Bayesian network. The results from both methods are combined to provide a final damage assessment.


In [16]:
# Function to map input values to states
def map_to_state(value, thresholds):
    if value <= thresholds[0]:
        return 0  # Minor
    elif value <= thresholds[1]:
        return 1  # Moderate
    else:
        return 2  # Severe

def evaluate_rules(crack_width_input, leaning_angle_input, roof_sagging_input,
                   foundation_cracks_input, fire_level_input):
    # Fuzzy Logic Evaluation
    damage_sim.input['crack_width'] = crack_width_input
    damage_sim.input['leaning_angle'] = leaning_angle_input
    damage_sim.input['roof_sagging'] = roof_sagging_input
    damage_sim.input['foundation_cracks'] = foundation_cracks_input
    damage_sim.input['fire_level'] = fire_level_input
    damage_sim.compute()
    fuzzy_damage = damage_sim.output['damage_severity']

    # Map input values to states for Bayesian Network
    crack_width_state = map_to_state(crack_width_input, [3, 10])
    leaning_angle_state = map_to_state(leaning_angle_input, [1, 2])
    roof_sagging_state = map_to_state(roof_sagging_input, [3, 8])
    foundation_cracks_state = map_to_state(foundation_cracks_input, [5, 10])
    fire_level_state = map_to_state(fire_level_input, [30, 70])

    # Probabilistic Reasoning Evaluation
    result = inference.query(
        variables=['Structural_Damage'],
        evidence={
            'Crack_Width': crack_width_state,
            'Leaning_Angle': leaning_angle_state,
            'Roof_Sagging': roof_sagging_state,
            'Foundation_Cracks': foundation_cracks_state,
            'Fire_Level': fire_level_state
        }
    )
    prob_damage = result.values

    # Map Bayesian Network states to scores
    state_to_score = {0: 2, 1: 5, 2: 8}

    # Combine Fuzzy and Probabilistic Results
    # Weighted average (50% fuzzy, 50% probabilistic)
    combined_score = 0.5 * fuzzy_damage + 0.5 * state_to_score[np.argmax(prob_damage)]

    # Determine the final damage level
    if combined_score <= 4:
        damage_level = "Minor Damage"
    elif combined_score <= 8:
        damage_level = "Moderate Damage"
    else:
        damage_level = "Severe Damage"

    return damage_level, fuzzy_damage, prob_damage, combined_score

# Run program and Test Cases


## Test Case 1: Minor Damage
Input:






*   Crack Width: 2 mm (Minor)


*   Leaning Angle: 0.5% (Minor)
*   Roof Sagging: 1 cm (Minor)



*   Foundation Cracks: 3 mm (Minor)

*   Fire Level: 20% (Minor)



In [17]:
print("=== Building Damage Assessment Expert System ===")
print("Enter the observed conditions:")

# Input observed conditions
crack_width_input = float(input("Crack Width (mm): "))
leaning_angle_input = float(input("Leaning Angle (%): "))
roof_sagging_input = float(input("Roof Sagging (cm): "))
foundation_cracks_input = float(input("Foundation Cracks (mm): "))
fire_level_input = float(input("Fire Level (%): "))

# Evaluate rules
damage_level, fuzzy_damage, prob_damage, combined_score = evaluate_rules(
    crack_width_input, leaning_angle_input, roof_sagging_input,
    foundation_cracks_input, fire_level_input
)

# Output results
print("\n=== Assessment Results ===")
print(f"Fuzzy Logic Damage Score: {fuzzy_damage:.2f}")
print(f"Probabilistic Reasoning Damage Probabilities: {prob_damage}")
print(f"Combined Damage Score: {combined_score:.2f}")
print(f"Final Damage Level: {damage_level}")

=== Building Damage Assessment Expert System ===
Enter the observed conditions:
Crack Width (mm): 2
Leaning Angle (%): 0.5
Roof Sagging (cm): 1
Foundation Cracks (mm): 3
Fire Level (%): 20

=== Assessment Results ===
Fuzzy Logic Damage Score: 1.69
Probabilistic Reasoning Damage Probabilities: [0.8  0.15 0.05]
Combined Damage Score: 1.84
Final Damage Level: Minor Damage


**Forward Chaining:**


1.   Fuzzy logic assigns low damage scores based on minor levels for all inputs.
2.   Bayesian inference determines the highest probability for the "Minor Damage" state because all evidence points to minimal structural stress.
3.  Combined score reflects a predominantly "Minor Damage" level.







**Backward Chaining:**

Goal: Determine if the damage is Minor.

Evidence: Most input values support the "Minor" category.

Conclusion: The hypothesis is supported.

## Test Case 2: Moderate Damage
Input:
*   Crack Width: 6 mm (Moderate)
*   Leaning Angle: 1.5% (Moderate)
*   Roof Sagging: 6 cm (Moderate)
*   Foundation Cracks: 7 mm (Moderate)
*   Fire Level: 50% (Moderate)




In [18]:
print("=== Building Damage Assessment Expert System ===")
print("Enter the observed conditions:")

# Input observed conditions
crack_width_input = float(input("Crack Width (mm): "))
leaning_angle_input = float(input("Leaning Angle (%): "))
roof_sagging_input = float(input("Roof Sagging (cm): "))
foundation_cracks_input = float(input("Foundation Cracks (mm): "))
fire_level_input = float(input("Fire Level (%): "))

# Evaluate rules
damage_level, fuzzy_damage, prob_damage, combined_score = evaluate_rules(
    crack_width_input, leaning_angle_input, roof_sagging_input,
    foundation_cracks_input, fire_level_input
)

# Output results
print("\n=== Assessment Results ===")
print(f"Fuzzy Logic Damage Score: {fuzzy_damage:.2f}")
print(f"Probabilistic Reasoning Damage Probabilities: {prob_damage}")
print(f"Combined Damage Score: {combined_score:.2f}")
print(f"Final Damage Level: {damage_level}")

=== Building Damage Assessment Expert System ===
Enter the observed conditions:
Crack Width (mm): 6
Leaning Angle (%): 1.5
Roof Sagging (cm): 6
Foundation Cracks (mm): 7
Fire Level (%): 50

=== Assessment Results ===
Fuzzy Logic Damage Score: 6.10
Probabilistic Reasoning Damage Probabilities: [0.1 0.3 0.6]
Combined Damage Score: 7.05
Final Damage Level: Moderate Damage


**Forward Chaining:**


1.   Fuzzy logic assigns low damage scores based on Moderate  levels for all inputs.
2.   Bayesian inference determines the highest probability for the "Moderate  Damage" state because all evidence points to minimal structural stress.
3.  Combined score reflects a predominantly "Moderate  Damage" level.







**Backward Chaining:**

* Goal: Determine if the damage is Moderate.

* Evidence: Most input values support the "Moderate" category.

* Conclusion: The hypothesis is supported.

## Test Case 3: Severe Damage
Input:
*   Crack Width: 12 mm (Severe)
*   Leaning Angle: 2.5% (Severe)
*   Roof Sagging: 9 cm (Severe)
*   Foundation Cracks: 12 mm (Severe)
*   Fire Level: 90% (Severe)




In [19]:
print("=== Building Damage Assessment Expert System ===")
print("Enter the observed conditions:")

# Input observed conditions
crack_width_input = float(input("Crack Width (mm): "))
leaning_angle_input = float(input("Leaning Angle (%): "))
roof_sagging_input = float(input("Roof Sagging (cm): "))
foundation_cracks_input = float(input("Foundation Cracks (mm): "))
fire_level_input = float(input("Fire Level (%): "))

# Evaluate rules
damage_level, fuzzy_damage, prob_damage, combined_score = evaluate_rules(
    crack_width_input, leaning_angle_input, roof_sagging_input,
    foundation_cracks_input, fire_level_input
)

# Output results
print("\n=== Assessment Results ===")
print(f"Fuzzy Logic Damage Score: {fuzzy_damage:.2f}")
print(f"Probabilistic Reasoning Damage Probabilities: {prob_damage}")
print(f"Combined Damage Score: {combined_score:.2f}")
print(f"Final Damage Level: {damage_level}")

=== Building Damage Assessment Expert System ===
Enter the observed conditions:
Crack Width (mm): 12
Leaning Angle (%): 2.5
Roof Sagging (cm): 9
Foundation Cracks (mm): 12
Fire Level (%): 90

=== Assessment Results ===
Fuzzy Logic Damage Score: 8.60
Probabilistic Reasoning Damage Probabilities: [0.1 0.3 0.6]
Combined Damage Score: 8.30
Final Damage Level: Severe Damage


**Forward Chaining:**


1.   Fuzzy logic assigns low damage scores based on Severe  levels for all inputs.
2.   Bayesian inference determines the highest probability for the "Severe  Damage" state because all evidence points to minimal structural stress.
3.  Combined score reflects a predominantly "Severe  Damage" level.







**Backward Chaining:**

* Goal: Determine if the damage is Severe.

* Evidence: Most input values support the "Severe" category.

* Conclusion: The hypothesis is supported.